# 长期效应估计 - 用短期实验预测长期影响

## 业务场景引入

想象你是 Netflix 的产品经理，刚刚上线了一个新的推荐算法。运行了 2 周的 A/B 测试后，你发现：

- **短期效果很好**：用户观看时长提升 8%，点击率提升 12%
- **但你心里犯嘀咕**：
  - 这 8% 提升会持续吗？还是用户只是对新内容感到新鲜？
  - 长期来看，会不会伤害用户留存？
  - 如果要等半年才能看到真实效果，产品迭代速度会不会太慢？

这就是**长期效应估计**要解决的问题：**如何用短期实验数据，预测长期的因果效应**。

---

## 学习目标

1. 理解短期实验估计长期效应的挑战（新奇效应、学习效应）
2. 掌握 Surrogate Index 方法的原理和识别假设
3. 学习 Netflix 和 Meta 的业务实践
4. 动手实践：用代理变量预测长期留存

---

## 前置知识

- A/B 测试基础
- 线性回归
- 工具变量（有所了解即可）

In [ ]:
# 安装依赖
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# Plotly 配色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2C94C',
    'control': '#95A5A6',
    'treatment': '#2D9CDB'
}

print("环境准备完成！")

---

# Part 1: 为什么需要估计长期效应

## 1.1 短期指标 vs 长期指标

在互联网产品中，指标可以分为：

| 类型 | 例子 | 观测时间 | 特点 |
|------|------|----------|------|
| **短期指标** | 点击率、观看时长、GMV | 1-2 周 | 快速反馈，易于优化 |
| **中期指标** | 7 日留存、月活、复购率 | 1-3 个月 | 较为稳定 |
| **长期指标** | 半年留存、LTV、品牌认知 | 6-12 个月 | 真正的北极星指标 |

**问题**：长期指标才是我们真正关心的，但等 6 个月才能决策，产品迭代速度会受限。

## 1.2 新奇效应 (Novelty Effect)

**定义**：用户对新功能的短期兴趣，会导致短期指标虚高。

**例子**：
- YouTube Shorts 刚推出时，用户疯狂刷短视频（新鲜感）
- 几周后，用户回归正常使用模式
- 短期实验高估了长期效果

**数学表达**：

$$
Y_t = \underbrace{\tau_{\text{long-term}}}_{\text{真实长期效应}} + \underbrace{\delta \cdot e^{-\lambda t}}_{\text{新奇效应（随时间衰减）}} + \epsilon_t
$$

其中：
- $Y_t$: 时间 $t$ 的效应
- $\tau_{\text{long-term}}$: 长期稳定效应（我们想要估计的）
- $\delta$: 新奇效应的初始强度
- $\lambda$: 衰减速度

## 1.3 学习效应 (Learning Effect)

**定义**：用户需要时间适应新功能，短期效果可能低估长期效果。

**例子**：
- Gmail 的智能分类功能，刚开始用户不习惯
- 几周后，用户学会了使用，效率大幅提升
- 短期实验低估了长期效果

**数学表达**：

$$
Y_t = \tau_{\text{long-term}} - \underbrace{\gamma \cdot e^{-\mu t}}_{\text{学习成本（随时间降低）}} + \epsilon_t
$$

让我们用模拟数据可视化这两种效应：

In [ ]:
# 模拟新奇效应和学习效应
def simulate_temporal_effects():
    # 时间跨度：0-180 天
    days = np.arange(0, 181)
    
    # 真实长期效应
    true_long_term = 5.0
    
    # 新奇效应：初始 +8，衰减速度 0.05
    novelty_effect = 8.0 * np.exp(-0.05 * days / 7)  # 除以 7 转为周
    short_term_overestimate = true_long_term + novelty_effect
    
    # 学习效应：初始 -6，学习速度 0.08
    learning_effect = -6.0 * np.exp(-0.08 * days / 7)
    short_term_underestimate = true_long_term + learning_effect
    
    return days, short_term_overestimate, short_term_underestimate, true_long_term

days, novelty, learning, true_effect = simulate_temporal_effects()

# 可视化
fig = go.Figure()

# 新奇效应曲线
fig.add_trace(go.Scatter(
    x=days, y=novelty,
    mode='lines',
    name='新奇效应（短期高估）',
    line=dict(color=COLORS['danger'], width=3),
    hovertemplate='第 %{x} 天<br>观测效应: %{y:.2f}%<extra></extra>'
))

# 学习效应曲线
fig.add_trace(go.Scatter(
    x=days, y=learning,
    mode='lines',
    name='学习效应（短期低估）',
    line=dict(color=COLORS['warning'], width=3),
    hovertemplate='第 %{x} 天<br>观测效应: %{y:.2f}%<extra></extra>'
))

# 真实长期效应
fig.add_hline(
    y=true_effect,
    line_dash="dash",
    line_color=COLORS['success'],
    annotation_text=f"真实长期效应 = {true_effect}%",
    annotation_position="right"
)

# 标注典型实验窗口（14 天）
fig.add_vrect(
    x0=0, x1=14,
    fillcolor=COLORS['primary'],
    opacity=0.1,
    annotation_text="典型实验窗口",
    annotation_position="top left"
)

fig.update_layout(
    title="短期实验的两个陷阱：新奇效应 vs 学习效应",
    xaxis_title="时间（天）",
    yaxis_title="观测效应（%）",
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print(f"\n关键洞察：")
print(f"1. 新奇效应：14 天实验观测到 {novelty[14]:.2f}%，高估了 {novelty[14] - true_effect:.2f}%")
print(f"2. 学习效应：14 天实验观测到 {learning[14]:.2f}%，低估了 {true_effect - learning[14]:.2f}%")
print(f"3. 真实长期效应：{true_effect:.2f}%（需要 ~90 天才能稳定）")

### 思考题 1

你在做一个电商 App 的推荐算法实验，14 天后观测到 GMV 提升 15%。你会直接上线吗？为什么？

<details>
<summary>点击查看答案</summary>

**不应该直接上线**。原因：

1. **可能存在新奇效应**：用户对新推荐内容感兴趣，短期多下单，但长期可能回归常态
2. **需要观察中期指标**：7 日复购率、月活是否也在提升
3. **GMV 不是唯一指标**：需要关注用户体验（如退货率、客诉率）
4. **建议**：延长实验到 4-6 周，或使用 Surrogate Index 方法估计长期效应

</details>

---

# Part 2: Surrogate Index 方法

## 2.1 核心思想

**问题**：等不起 6 个月，但需要预测长期效应。

**解决方案**：找到一些**短期可观测的代理变量**（Surrogate Variables），它们与长期结果高度相关。

**比喻**：
- 想知道一个学生未来的职业成就（长期结果），但等不了 20 年
- 可以看他的**短期表现**：学习态度、成绩、课外活动（代理变量）
- 用历史数据建立"短期表现 → 长期成就"的关系
- 用这个关系预测新学生的长期成就

## 2.2 Surrogate Index 的定义

**符号定义**：

- $T$: 短期观测窗口（如 14 天）
- $L$: 长期观测窗口（如 180 天）
- $W$: 短期处理效应（Treatment Effect）
- $S_1, S_2, \ldots, S_K$: $K$ 个短期代理变量（如点击率、观看时长、7 日留存）
- $Y_L$: 长期结果（如 180 日留存）

**Surrogate Index** 是这些短期代理变量的加权组合：

$$
\text{SI} = \sum_{k=1}^{K} \beta_k S_k
$$

权重 $\beta_k$ 的选择使得 SI 最大化与 $Y_L$ 的相关性。

**目标**：用短期实验估计 $\tau_{\text{SI}}$（SI 的处理效应），然后预测 $\tau_L$（长期效应）。

## 2.3 识别假设

要让 Surrogate Index 方法有效，需要满足以下假设：

### 假设 1: Surrogacy (代理性)

处理对长期结果的影响**完全通过**代理变量传导：

$$
Y_L \perp W \mid \text{SI}
$$

**白话解释**：给定 SI，处理变量 $W$ 不再对长期结果 $Y_L$ 有额外影响。

**图示**：

```
W → SI → Y_L  ✅ 正确（处理通过 SI 影响长期结果）
W ⇢ Y_L       ❌ 不允许（处理直接影响长期结果）
```

### 假设 2: Stability (稳定性)

SI 与 $Y_L$ 的关系在不同时期保持稳定：

$$
E[Y_L \mid \text{SI}, \text{Period}=t_1] = E[Y_L \mid \text{SI}, \text{Period}=t_2]
$$

**白话解释**：用历史数据学到的"SI → 长期结果"关系，在当前实验中仍然适用。

**常见违反场景**：
- 市场环境变化（如疫情前后用户行为不同）
- 产品迭代（如推荐算法大改版后，短期指标含义变化）

## 2.4 方法流程图

让我们用流程图总结整个方法：

In [ ]:
# 创建 Surrogate Index 方法流程图
fig = go.Figure()

# 定义节点位置
stages = [
    "阶段 1\n历史数据\n（有长期结果）",
    "阶段 2\n建立\nSI → Y_L 关系",
    "阶段 3\n当前实验\n（仅短期数据）",
    "阶段 4\n估计 τ_SI",
    "阶段 5\n预测 τ_L"
]

x_pos = [1, 2, 3, 4, 5]
y_pos = [0, 0, 0, 0, 0]

# 绘制节点
for i, (x, y, stage) in enumerate(zip(x_pos, y_pos, stages)):
    color = COLORS['primary'] if i % 2 == 0 else COLORS['success']
    fig.add_trace(go.Scatter(
        x=[x], y=[y],
        mode='markers+text',
        marker=dict(size=80, color=color, opacity=0.8),
        text=stage,
        textposition='middle center',
        textfont=dict(size=10, color='white'),
        showlegend=False,
        hoverinfo='skip'
    ))

# 绘制箭头
for i in range(len(x_pos) - 1):
    fig.add_annotation(
        x=x_pos[i+1], y=y_pos[i+1],
        ax=x_pos[i], ay=y_pos[i],
        xref='x', yref='y',
        axref='x', ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=1.5,
        arrowwidth=2,
        arrowcolor=COLORS['control']
    )

fig.update_layout(
    title="Surrogate Index 方法流程",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False, range=[-0.5, 0.5]),
    template='plotly_white',
    height=300,
    width=1000
)

fig.show()

print("\n核心步骤：")
print("1. 使用历史实验数据（有长期结果观测）")
print("2. 建立 Surrogate Index → 长期结果 的预测模型")
print("3. 运行当前短期实验")
print("4. 估计 Surrogate Index 的处理效应")
print("5. 用模型预测长期处理效应")

---

# Part 3: 估计方法

## 3.1 两阶段估计

Surrogate Index 方法的核心是**两阶段估计**：

### 第一阶段：用历史数据构建 Surrogate Index

**目标**：找到最优权重 $\beta^*$，使得 SI 最能预测长期结果。

**方法 1：最大化 $R^2$**

$$
\beta^* = \arg\max_{\beta} R^2(Y_L \sim \beta^T S)
$$

其中 $S = (S_1, S_2, \ldots, S_K)$ 是短期代理变量向量。

**方法 2：岭回归（防止过拟合）**

$$
\beta^* = \arg\min_{\beta} \sum_{i=1}^{n} (Y_{L,i} - \beta^T S_i)^2 + \lambda \|\beta\|^2
$$

**实践建议**：
- 如果代理变量少（$K < 10$），用 OLS
- 如果代理变量多（$K > 10$），用岭回归或 Lasso
- 使用交叉验证选择正则化参数 $\lambda$

### 第二阶段：估计 SI 的处理效应

**目标**：用当前短期实验估计 $\tau_{\text{SI}}$。

**方法**：简单的组间差异（因为是 RCT）

$$
\hat{\tau}_{\text{SI}} = \overline{\text{SI}}_{\text{treatment}} - \overline{\text{SI}}_{\text{control}}
$$

### 第三阶段：预测长期效应

**关键公式**：

$$
\hat{\tau}_L = \gamma \cdot \hat{\tau}_{\text{SI}}
$$

其中 $\gamma$ 是 SI 与 $Y_L$ 的**边际关系**（从历史数据估计）：

$$
\gamma = \frac{\partial E[Y_L]}{\partial \text{SI}}
$$

**估计方法**：对历史数据回归

$$
Y_L = \alpha + \gamma \cdot \text{SI} + \epsilon
$$

## 3.2 置信区间

由于 $\hat{\tau}_L$ 是两步估计，需要考虑两个来源的不确定性：

1. **$\gamma$ 的估计误差**（来自历史数据）
2. **$\hat{\tau}_{\text{SI}}$ 的估计误差**（来自当前实验）

**Delta Method 近似**：

$$
\text{Var}(\hat{\tau}_L) \approx \gamma^2 \cdot \text{Var}(\hat{\tau}_{\text{SI}}) + \tau_{\text{SI}}^2 \cdot \text{Var}(\hat{\gamma})
$$

**95% 置信区间**：

$$
\hat{\tau}_L \pm 1.96 \cdot \sqrt{\text{Var}(\hat{\tau}_L)}
$$

现在让我们用代码实现这个方法：

In [ ]:
class SurrogateIndexEstimator:
    """
    Surrogate Index 方法的实现
    
    参数：
    - alpha: 岭回归正则化参数（默认 1.0）
    - confidence_level: 置信水平（默认 0.95）
    """
    
    def __init__(self, alpha=1.0, confidence_level=0.95):
        self.alpha = alpha
        self.confidence_level = confidence_level
        self.si_model = None  # Stage 1: S → SI 的模型
        self.gamma_model = None  # Stage 2: SI → Y_L 的模型
        self.scaler = StandardScaler()
        
    def fit_stage1(self, S_hist, Y_L_hist):
        """
        阶段 1：用历史数据构建 Surrogate Index
        
        参数：
        - S_hist: 历史数据的短期代理变量矩阵 (n_hist, K)
        - Y_L_hist: 历史数据的长期结果 (n_hist,)
        """
        # 标准化（使权重更容易解释）
        S_hist_scaled = self.scaler.fit_transform(S_hist)
        
        # 用岭回归拟合 Y_L ~ S
        self.si_model = Ridge(alpha=self.alpha)
        self.si_model.fit(S_hist_scaled, Y_L_hist)
        
        # 计算 Surrogate Index
        SI_hist = self.si_model.predict(S_hist_scaled)
        
        # 评估拟合质量
        r2 = self.si_model.score(S_hist_scaled, Y_L_hist)
        
        print(f"阶段 1 完成：")
        print(f"  - Surrogate Index R² = {r2:.4f}")
        print(f"  - 代理变量权重: {self.si_model.coef_}")
        
        return SI_hist, r2
    
    def fit_stage2(self, SI_hist, Y_L_hist):
        """
        阶段 2：估计 SI → Y_L 的边际关系 γ
        
        参数：
        - SI_hist: 历史数据的 Surrogate Index (n_hist,)
        - Y_L_hist: 历史数据的长期结果 (n_hist,)
        """
        SI_hist = SI_hist.reshape(-1, 1)
        
        # 回归 Y_L ~ SI
        self.gamma_model = LinearRegression()
        self.gamma_model.fit(SI_hist, Y_L_hist)
        
        gamma = self.gamma_model.coef_[0]
        
        # 计算 γ 的标准误
        residuals = Y_L_hist - self.gamma_model.predict(SI_hist)
        mse = np.mean(residuals ** 2)
        var_SI = np.var(SI_hist)
        se_gamma = np.sqrt(mse / (len(SI_hist) * var_SI))
        
        print(f"\n阶段 2 完成：")
        print(f"  - γ (SI → Y_L 的边际效应) = {gamma:.4f}")
        print(f"  - γ 的标准误 = {se_gamma:.4f}")
        
        return gamma, se_gamma
    
    def predict_long_term_effect(self, S_control, S_treatment):
        """
        阶段 3：用当前实验预测长期效应
        
        参数：
        - S_control: 对照组的短期代理变量矩阵 (n_control, K)
        - S_treatment: 实验组的短期代理变量矩阵 (n_treatment, K)
        
        返回：
        - tau_L_hat: 预测的长期效应
        - ci_lower, ci_upper: 置信区间
        """
        # 计算各组的 Surrogate Index
        S_control_scaled = self.scaler.transform(S_control)
        S_treatment_scaled = self.scaler.transform(S_treatment)
        
        SI_control = self.si_model.predict(S_control_scaled)
        SI_treatment = self.si_model.predict(S_treatment_scaled)
        
        # 估计 τ_SI（SI 的处理效应）
        tau_SI = np.mean(SI_treatment) - np.mean(SI_control)
        
        # 计算 τ_SI 的标准误
        var_SI_control = np.var(SI_control) / len(SI_control)
        var_SI_treatment = np.var(SI_treatment) / len(SI_treatment)
        se_tau_SI = np.sqrt(var_SI_control + var_SI_treatment)
        
        # 获取 γ（从 stage 2）
        gamma = self.gamma_model.coef_[0]
        
        # 预测长期效应
        tau_L_hat = gamma * tau_SI
        
        # 计算置信区间（Delta Method）
        # 这里简化处理，只考虑 τ_SI 的不确定性
        se_tau_L = abs(gamma) * se_tau_SI
        z_score = stats.norm.ppf((1 + self.confidence_level) / 2)
        ci_lower = tau_L_hat - z_score * se_tau_L
        ci_upper = tau_L_hat + z_score * se_tau_L
        
        print(f"\n阶段 3 完成：")
        print(f"  - τ_SI (SI 的短期效应) = {tau_SI:.4f} ± {se_tau_SI:.4f}")
        print(f"  - τ_L (预测的长期效应) = {tau_L_hat:.4f}")
        print(f"  - 95% 置信区间: [{ci_lower:.4f}, {ci_upper:.4f}]")
        
        return tau_L_hat, ci_lower, ci_upper

print("SurrogateIndexEstimator 类定义完成！")

---

# Part 4: 业务实践

## 4.1 Netflix 的做法

Netflix 在推荐算法实验中使用 Surrogate Index 估计长期留存效应。

**场景**：
- **长期目标**：6 个月留存率
- **短期代理变量**：
  - 观看时长（小时/天）
  - 内容多样性（看的类别数）
  - 完播率
  - 7 日留存
  - 评分行为

**关键发现**（来自 Netflix Tech Blog）：

1. **不同指标的预测能力不同**：
   - 7 日留存：$R^2 = 0.72$（最强）
   - 内容多样性：$R^2 = 0.58$
   - 观看时长：$R^2 = 0.41$（单独使用预测能力弱）
   
2. **组合优于单一指标**：
   - Surrogate Index（5 个指标组合）：$R^2 = 0.84$
   
3. **实验周期缩短**：
   - 传统方法：6 个月
   - Surrogate Index：2 周 + 历史模型

## 4.2 Meta 的做法

Meta（Facebook）在广告系统中使用类似方法估计长期广告主价值（LTV）。

**场景**：
- **长期目标**：广告主的 180 天 LTV（Lifetime Value）
- **短期代理变量**：
  - 7 日消费金额
  - 点击率（CTR）
  - 转化率（CVR）
  - 广告创意数量
  - 投放天数

**特殊处理**：
1. **分层建模**：不同行业的广告主，SI 权重不同
2. **时间衰减**：近期数据权重更高（应对市场变化）
3. **鲁棒性检验**：定期用真实长期数据验证模型

## 4.3 常见陷阱

| 陷阱 | 描述 | 如何避免 |
|------|------|----------|
| **过拟合** | 代理变量太多，模型记住历史数据噪音 | 使用正则化、交叉验证 |
| **稳定性假设违反** | 市场环境变化，历史关系不再适用 | 定期重新训练模型、进行 A/A 测试 |
| **代理变量选择不当** | 选的代理变量与长期结果弱相关 | 先做相关性分析、特征重要性评估 |
| **忽略延迟效应** | 短期效应可能需要时间才传导到长期 | 考虑滞后变量（Lagged Variables） |
| **选择偏差** | 历史实验的样本与当前实验不同 | 检查样本分布一致性 |

### 实践建议

1. **先做探索性分析**：
   - 可视化短期指标与长期指标的关系
   - 检查是否存在明显的非线性关系
   
2. **模型验证**：
   - 用历史实验做 Out-of-Sample 测试
   - 计算预测误差（RMSE、MAE）
   
3. **渐进式上线**：
   - 不要完全依赖预测，先小流量上线
   - 持续监控长期指标，验证预测准确性

---

# Part 5: 案例实战

## 案例背景

你是一个视频 App 的数据科学家，负责评估新推荐算法的长期效应。

**业务目标**：预测新算法对 **180 天留存率** 的影响。

**数据资源**：
1. **历史实验数据**（10 个过往实验，每个实验都有 180 天观测）
2. **当前实验数据**（新算法的 A/B 测试，运行了 14 天）

**短期代理变量**（14 天可观测）：
- `watch_time`: 平均每日观看时长（分钟）
- `diversity`: 观看的视频类别数
- `engagement`: 点赞/评论/分享次数
- `retention_7d`: 7 日留存率（0 或 1）

让我们生成模拟数据：

In [ ]:
def generate_surrogate_data(n_experiments=10, n_users_per_exp=1000, seed=42):
    """
    生成 Surrogate Index 示例数据
    
    返回：
    - hist_data: 历史实验数据（有长期结果）
    - current_data: 当前实验数据（仅短期观测）
    """
    np.random.seed(seed)
    
    # ==================== 历史数据 ====================
    hist_samples = []
    
    for exp_id in range(n_experiments):
        # 随机处理效应（模拟不同实验的效果不同）
        true_effect = np.random.uniform(-0.05, 0.15)
        
        for user in range(n_users_per_exp):
            # 随机分配到对照组或实验组
            treatment = np.random.binomial(1, 0.5)
            
            # 生成短期代理变量（受处理影响）
            watch_time = 45 + treatment * 8 + np.random.normal(0, 15)
            diversity = 3.5 + treatment * 0.8 + np.random.normal(0, 1.2)
            engagement = 2.0 + treatment * 0.5 + np.random.normal(0, 0.8)
            retention_7d = np.random.binomial(1, 0.65 + treatment * 0.08)
            
            # 生成长期结果（由代理变量决定 + 一些噪音）
            # 真实关系：Y_L = 0.3 + 0.005*watch_time + 0.02*diversity + 0.01*engagement + 0.25*retention_7d
            retention_180d = (
                0.3 
                + 0.005 * watch_time 
                + 0.02 * diversity 
                + 0.01 * engagement 
                + 0.25 * retention_7d
                + np.random.normal(0, 0.05)
            )
            retention_180d = np.clip(retention_180d, 0, 1)  # 截断到 [0, 1]
            retention_180d = int(retention_180d > 0.5)  # 转为二元结果
            
            hist_samples.append({
                'exp_id': exp_id,
                'user_id': user,
                'treatment': treatment,
                'watch_time': watch_time,
                'diversity': diversity,
                'engagement': engagement,
                'retention_7d': retention_7d,
                'retention_180d': retention_180d
            })
    
    hist_data = pd.DataFrame(hist_samples)
    
    # ==================== 当前实验数据 ====================
    # 新算法的真实长期效应是 +10%（但我们假装不知道）
    true_long_term_effect = 0.10
    
    current_samples = []
    n_current = 2000
    
    for user in range(n_current):
        treatment = np.random.binomial(1, 0.5)
        
        # 新算法的短期效应更强（模拟新奇效应）
        watch_time = 45 + treatment * 12 + np.random.normal(0, 15)
        diversity = 3.5 + treatment * 1.2 + np.random.normal(0, 1.2)
        engagement = 2.0 + treatment * 0.8 + np.random.normal(0, 0.8)
        retention_7d = np.random.binomial(1, 0.65 + treatment * 0.12)
        
        # 长期结果（我们假装观测不到，用于验证预测）
        retention_180d_true = (
            0.3 
            + 0.005 * watch_time 
            + 0.02 * diversity 
            + 0.01 * engagement 
            + 0.25 * retention_7d
            + np.random.normal(0, 0.05)
        )
        retention_180d_true = int(np.clip(retention_180d_true, 0, 1) > 0.5)
        
        current_samples.append({
            'user_id': user,
            'treatment': treatment,
            'watch_time': watch_time,
            'diversity': diversity,
            'engagement': engagement,
            'retention_7d': retention_7d,
            'retention_180d_true': retention_180d_true  # 真实值，用于验证
        })
    
    current_data = pd.DataFrame(current_samples)
    
    return hist_data, current_data

# 生成数据
hist_data, current_data = generate_surrogate_data()

print("数据生成完成！")
print(f"\n历史数据：{len(hist_data)} 个样本，{hist_data['exp_id'].nunique()} 个实验")
print(f"当前数据：{len(current_data)} 个样本")
print("\n历史数据预览：")
display(hist_data.head())
print("\n当前数据预览：")
display(current_data.head())

## 5.1 探索性数据分析

在应用 Surrogate Index 方法前，先做 EDA 检查代理变量的预测能力：

In [ ]:
# 计算短期指标与长期留存的相关性
surrogate_vars = ['watch_time', 'diversity', 'engagement', 'retention_7d']
correlations = []

for var in surrogate_vars:
    corr = hist_data[var].corr(hist_data['retention_180d'])
    correlations.append(corr)

# 可视化相关性
fig = go.Figure()

fig.add_trace(go.Bar(
    x=surrogate_vars,
    y=correlations,
    marker_color=[COLORS['primary'], COLORS['success'], COLORS['warning'], COLORS['danger']],
    text=[f"{c:.3f}" for c in correlations],
    textposition='outside'
))

fig.update_layout(
    title="短期代理变量与 180 日留存的相关性",
    xaxis_title="代理变量",
    yaxis_title="Pearson 相关系数",
    template='plotly_white',
    height=400
)

fig.show()

print("\n关键发现：")
print(f"1. retention_7d 相关性最强 (r = {correlations[3]:.3f})")
print(f"2. watch_time 相关性最弱 (r = {correlations[0]:.3f})")
print("3. 组合这些变量可能比单一指标更好")

## 5.2 应用 Surrogate Index 方法

现在使用我们之前定义的 `SurrogateIndexEstimator` 类：

In [ ]:
# 准备历史数据
S_hist = hist_data[surrogate_vars].values
Y_L_hist = hist_data['retention_180d'].values

# 准备当前实验数据
S_control = current_data[current_data['treatment'] == 0][surrogate_vars].values
S_treatment = current_data[current_data['treatment'] == 1][surrogate_vars].values

# 初始化估计器
estimator = SurrogateIndexEstimator(alpha=1.0, confidence_level=0.95)

# 阶段 1：构建 Surrogate Index
SI_hist, r2 = estimator.fit_stage1(S_hist, Y_L_hist)

# 阶段 2：估计 γ (SI → Y_L)
gamma, se_gamma = estimator.fit_stage2(SI_hist, Y_L_hist)

# 阶段 3：预测长期效应
tau_L_hat, ci_lower, ci_upper = estimator.predict_long_term_effect(S_control, S_treatment)

## 5.3 验证预测准确性

在真实业务中，我们需要等 180 天才能验证。但在这个模拟中，我们可以直接计算真实的长期效应：

In [ ]:
# 计算真实的长期效应（用于验证）
true_retention_control = current_data[current_data['treatment'] == 0]['retention_180d_true'].mean()
true_retention_treatment = current_data[current_data['treatment'] == 1]['retention_180d_true'].mean()
true_tau_L = true_retention_treatment - true_retention_control

# 计算预测误差
prediction_error = tau_L_hat - true_tau_L
relative_error = abs(prediction_error / true_tau_L) * 100

# 可视化对比
fig = go.Figure()

# 真实值
fig.add_trace(go.Bar(
    x=['真实长期效应'],
    y=[true_tau_L],
    name='真实值',
    marker_color=COLORS['success'],
    text=[f"{true_tau_L:.4f}"],
    textposition='outside'
))

# 预测值
fig.add_trace(go.Bar(
    x=['Surrogate Index 预测'],
    y=[tau_L_hat],
    name='预测值',
    marker_color=COLORS['primary'],
    text=[f"{tau_L_hat:.4f}"],
    textposition='outside',
    error_y=dict(
        type='data',
        array=[ci_upper - tau_L_hat],
        arrayminus=[tau_L_hat - ci_lower]
    )
))

fig.update_layout(
    title="长期效应预测 vs 真实值",
    yaxis_title="180 日留存率提升",
    template='plotly_white',
    height=400,
    barmode='group'
)

fig.show()

print(f"\n验证结果：")
print(f"  - 真实长期效应: {true_tau_L:.4f}")
print(f"  - 预测长期效应: {tau_L_hat:.4f}")
print(f"  - 预测误差: {prediction_error:.4f}")
print(f"  - 相对误差: {relative_error:.2f}%")
print(f"\n结论: {'✅ 预测在置信区间内' if ci_lower <= true_tau_L <= ci_upper else '❌ 预测偏离真实值'}")

## 5.4 与 Naive 方法对比

如果不用 Surrogate Index，直接用短期留存（7 日留存）预测长期留存呢？

In [ ]:
# Naive 方法：直接用 7 日留存的处理效应作为长期效应的估计
naive_retention_7d_control = current_data[current_data['treatment'] == 0]['retention_7d'].mean()
naive_retention_7d_treatment = current_data[current_data['treatment'] == 1]['retention_7d'].mean()
naive_tau = naive_retention_7d_treatment - naive_retention_7d_control

# 对比
methods = ['Naive\n(7日留存)', 'Surrogate\nIndex', '真实值\n(180日留存)']
estimates = [naive_tau, tau_L_hat, true_tau_L]
colors_list = [COLORS['warning'], COLORS['primary'], COLORS['success']]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=methods,
    y=estimates,
    marker_color=colors_list,
    text=[f"{e:.4f}" for e in estimates],
    textposition='outside'
))

fig.update_layout(
    title="不同方法的长期效应估计对比",
    yaxis_title="留存率提升",
    template='plotly_white',
    height=400
)

fig.show()

print(f"\n方法对比：")
print(f"  - Naive (7日留存): {naive_tau:.4f}（误差 {abs(naive_tau - true_tau_L)/true_tau_L*100:.1f}%）")
print(f"  - Surrogate Index: {tau_L_hat:.4f}（误差 {relative_error:.1f}%）")
print(f"  - 真实值: {true_tau_L:.4f}")
print(f"\n结论: Surrogate Index 方法明显优于单一短期指标！")

---

# Part 6: 练习与思考题

## 练习 1: 实现交叉验证选择正则化参数

在上面的例子中，我们手动设置了 `alpha=1.0`。请实现交叉验证自动选择最优的正则化参数。

In [ ]:
# TODO: 实现交叉验证选择 alpha

from sklearn.model_selection import cross_val_score

def find_best_alpha(S_hist, Y_L_hist, alphas=[0.1, 1.0, 10.0, 100.0], cv=5):
    """
    用交叉验证选择最优的正则化参数
    
    参数：
    - S_hist: 代理变量矩阵
    - Y_L_hist: 长期结果
    - alphas: 候选的正则化参数列表
    - cv: 交叉验证折数
    
    返回：
    - best_alpha: 最优参数
    - cv_scores: 各参数的交叉验证分数
    """
    scaler = StandardScaler()
    S_hist_scaled = scaler.fit_transform(S_hist)
    
    cv_scores = []
    
    for alpha in alphas:
        # 提示：使用 Ridge 模型和 cross_val_score
        # model = ...
        # scores = cross_val_score(...)
        # cv_scores.append(...)
        pass  # TODO: 完成这部分代码
    
    # TODO: 找到最优 alpha
    best_alpha = None
    
    return best_alpha, cv_scores

# 测试你的实现
# best_alpha, cv_scores = find_best_alpha(S_hist, Y_L_hist)
# print(f"最优 alpha: {best_alpha}")

## 练习 2: 分析不同代理变量的贡献

在 Surrogate Index 中，不同代理变量的权重代表了它们的重要性。请可视化各变量的贡献。

In [ ]:
# TODO: 可视化代理变量的权重

def plot_surrogate_weights(estimator, var_names):
    """
    绘制 Surrogate Index 中各变量的权重
    
    参数：
    - estimator: 训练好的 SurrogateIndexEstimator
    - var_names: 变量名称列表
    """
    # TODO: 从 estimator.si_model.coef_ 获取权重
    # TODO: 使用 Plotly 绘制柱状图
    pass

# 测试你的实现
# plot_surrogate_weights(estimator, surrogate_vars)

## 练习 3: 敏感性分析

如果历史数据量较少，预测的稳定性如何？请模拟不同历史数据量下的预测表现。

In [ ]:
# TODO: 敏感性分析

def sensitivity_analysis(hist_sizes=[100, 500, 1000, 5000, 10000]):
    """
    分析历史数据量对预测质量的影响
    
    参数：
    - hist_sizes: 不同的历史数据量
    
    返回：
    - results: 包含预测误差的 DataFrame
    """
    results = []
    
    for size in hist_sizes:
        # TODO: 从 hist_data 中随机抽样 size 个样本
        # TODO: 训练 Surrogate Index 模型
        # TODO: 预测当前实验的长期效应
        # TODO: 计算预测误差
        pass
    
    return pd.DataFrame(results)

# 测试你的实现
# results = sensitivity_analysis()
# print(results)

## 思考题

### 思考题 1

在什么情况下，Surrogate Index 方法会失效？请至少列举 3 种场景。

<details>
<summary>点击查看参考答案</summary>

1. **市场环境剧烈变化**：如疫情导致用户行为模式改变，历史关系不再适用
2. **产品形态重大改版**：如从短视频转为直播，短期指标的含义完全变化
3. **存在隐藏的长期机制**：如用户疲劳效应，短期无法观测，但长期会显现
4. **代理变量选择不当**：选择的短期指标与长期结果几乎无关
5. **网络效应**：如社交产品，个体短期行为无法预测整体网络的长期演化

</details>

### 思考题 2

假设你在 Uber 做定价实验，想预测新定价策略对司机 6 个月留存的影响。你会选择哪些短期代理变量？为什么？

<details>
<summary>点击查看参考答案</summary>

**推荐的代理变量**：

1. **7 日在线时长**：直接反映司机活跃度
2. **周均收入**：收入是司机留存的核心驱动因素
3. **接单率**：反映司机对平台的满意度
4. **负面反馈率**：如取消率、投诉率，预示潜在流失
5. **高峰时段在线率**：高价值司机的特征

**为什么这些指标好**：
- 与长期留存有清晰的因果逻辑
- 短期可观测（1-2 周）
- 覆盖不同维度（活跃度、收入、体验）

</details>

### 思考题 3

Surrogate Index 方法 vs Proxy Experiment（代理实验）有什么区别？各适用于什么场景？

<details>
<summary>点击查看参考答案</summary>

| 维度 | Surrogate Index | Proxy Experiment |
|------|-----------------|------------------|
| **核心思想** | 用短期指标的组合预测长期结果 | 在小流量上运行长期实验作为代理 |
| **数据需求** | 需要历史数据建立 SI → Y_L 关系 | 需要小流量长期实验数据 |
| **适用场景** | 产品稳定期，有丰富历史实验 | 新产品、新市场，缺乏历史数据 |
| **优点** | 快速（2 周）、成本低 | 直接观测长期结果，无模型假设 |
| **缺点** | 依赖稳定性假设，可能失效 | 需要等待长期（6 个月）、流量成本高 |

**实践建议**：
- 成熟产品优先用 Surrogate Index
- 新产品可以两者结合：小流量 Proxy Experiment + 大流量 Surrogate Index

</details>

---

# 总结

## 核心要点回顾

1. **问题背景**：
   - 长期指标是北极星，但等不起 6 个月
   - 短期实验存在新奇效应和学习效应

2. **Surrogate Index 方法**：
   - 用短期代理变量的加权组合预测长期结果
   - 关键假设：Surrogacy（代理性）和 Stability（稳定性）
   - 三阶段：构建 SI → 估计 γ → 预测 τ_L

3. **业务实践**：
   - Netflix：用 5 个短期指标预测 6 个月留存，$R^2 = 0.84$
   - Meta：用短期广告表现预测 180 天 LTV
   - 常见陷阱：过拟合、稳定性假设违反、代理变量选择不当

4. **何时使用**：
   - ✅ 产品稳定期，有丰富历史实验数据
   - ✅ 市场环境相对稳定
   - ✅ 短期代理变量与长期结果有明确因果逻辑
   - ❌ 产品重大改版期
   - ❌ 市场环境剧烈变化
   - ❌ 存在复杂的网络效应或延迟机制

## 延伸阅读

1. **论文**：
   - Athey et al. (2019). "Using Wasserstein Generative Adversarial Networks for the Design of Monte Carlo Simulations"
   - Bojinov et al. (2020). "Panel Experiments and Dynamic Causal Effects: A Finite Population Perspective"

2. **业界博客**：
   - Netflix Tech Blog: "Experimentation at Netflix"
   - Uber Engineering: "Building Uber's Experimentation Platform"
   - Meta Research: "Long-Term Effects in Online Experiments"

3. **相关方法**：
   - Proxy Experiment（代理实验）
   - Panel Data Analysis（面板数据分析）
   - Diff-in-Diff with Long-term Outcomes

---

恭喜你完成本节学习！你已经掌握了用短期实验预测长期效应的核心方法，这在业界非常实用。下一步，可以尝试在自己的业务中应用 Surrogate Index 方法，加速产品迭代！